In [1]:
import requests
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

import folium

print("Libraries imported.")

Libraries imported.


In [99]:
CLIENT_ID = 'LOTOXYVKBNWICQKGW3RLTMUL50VO2EGVDS3S20AWIOZXTVZU' 
CLIENT_SECRET = 'V4ZZ2RVEIMMQVMU1VTUVIYVCLLTDWE1F5SSYG3WQYJHKE23B'
VERSION = '20180604'
LIMIT = 200

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LOTOXYVKBNWICQKGW3RLTMUL50VO2EGVDS3S20AWIOZXTVZU
CLIENT_SECRET:V4ZZ2RVEIMMQVMU1VTUVIYVCLLTDWE1F5SSYG3WQYJHKE23B


In [160]:
address = 'Mountain View'
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
print(location)

Mountain View, Santa Clara County, California, United States of America


In [161]:
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.3893889 -122.0832101


In [162]:
radius = 300

Exploring the area

In [163]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=LOTOXYVKBNWICQKGW3RLTMUL50VO2EGVDS3S20AWIOZXTVZU&client_secret=V4ZZ2RVEIMMQVMU1VTUVIYVCLLTDWE1F5SSYG3WQYJHKE23B&ll=37.3893889,-122.0832101&v=20180604&radius=300&limit=200'

In [164]:
request = requests.get(url).json()


In [165]:
request

{'meta': {'code': 200, 'requestId': '5e432fd51e152c001b12caa0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Mountain View',
  'headerFullLocation': 'Downtown Mountain View, Mountain View',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 37.3920889027, 'lng': -122.07981819365364},
   'sw': {'lat': 37.3866888973, 'lng': -122.08660200634637}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a036cfaefa82a5f1dd82b5a',
       'name': '1 Oz Coffee',
       'location': {'address': '130 Castro St',
        'lat': 37.38791722809527,
        'lng': -122.0828916224752,
        'label

In [166]:
items = request['response']['groups'][0]['items']
items

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5a036cfaefa82a5f1dd82b5a',
   'name': '1 Oz Coffee',
   'location': {'address': '130 Castro St',
    'lat': 37.38791722809527,
    'lng': -122.0828916224752,
    'labeledLatLngs': [{'label': 'display',
      'lat': 37.38791722809527,
      'lng': -122.0828916224752}],
    'distance': 166,
    'postalCode': '94041',
    'cc': 'US',
    'city': 'Mountain View',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['130 Castro St',
     'Mountain View, CA 94041',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1e0931735',
     'name': 'Coffee Shop',
     'pluralName': 'Coffee Shops',
     'shortName': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},

In [167]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [168]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', ] 
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)
# print(dataframe_filtered.shape)

,name,categories,lat,lng
0,1 Oz Coffee,Coffee Shop,37.387917,-122.082892
1,Pioneer Park,Park,37.389459,-122.083468
2,Eagle Park,Park,37.389046,-122.085656
3,Mountain View Optometry,Optical Shop,37.390168,-122.080933
4,Mountain View Center for the Performing Arts,Concert Hall,37.389937,-122.081725
5,Yoga Belly,Yoga Studio,37.390493,-122.080510
6,sweetgreen,Salad Place,37.390901,-122.081145
7,Mediterranean Grill House,Mediterranean Restaurant,37.387720,-122.082852
8,Masa Sushi,Sushi Restaurant,37.388225,-122.082493
9,Yoga is Youthfulness,Yoga Studio,37.388901,-122.082633


In [169]:
dataframe_filtered.shape

(23, 4)

In [170]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='San Jose, CA',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map